# Tutorial

## Overview

Ray is a Python-based distributed execution $\bf engine$. The same code can be run on a single machine to achieve efficient multiprocessing, and it can be used on a cluster for large computations.

When using Ray, several processes are involved.

   • Multiple $\bf worker$ processes execute tasks and store results in object stores. Each worker is a separate process.

   • One $\bf \text{object store}$ per node stores immutable objects in shared memory and allows workers to efficiently share objects on the same node with minimal copying and deserialization.

   • One $\bf \text{local scheduler}$ per node assigns tasks to workers on the same node.

   • A $\bf \text{global scheduler}$ receives tasks from local schedulers and assigns them to other local schedulers.

   • A $\bf \text{driver}$ is the Python process that the user controls. For example, if the user is running a script or using a Python shell, then the driver is the Python process that runs the script or the shell. A driver is similar to a worker in that it can submit tasks to its local scheduler and get objects from the object store, but it is different in that the local scheduler will not assign tasks to the driver to be executed.

   • A $\bf \text{Redis}$ server maintains much of the system’s state. For example, it keeps track of which objects live on which machines and of the task specifications (but not data). It can also be queried directly for debugging purposes.
   
 ## Starting Ray
 To start Ray, start Python and run the following commands.

In [1]:
import ray
ray.init()

Waiting for redis server at 127.0.0.1:28963 to respond...
Waiting for redis server at 127.0.0.1:17657 to respond...
Starting local scheduler with 8 CPUs, 0 GPUs

View the web UI at http://localhost:8889/notebooks/ray_ui44856.ipynb?token=b6270b2019013e9740412c7ef31ccf7c6ad9ad7b147bc2f5



{'local_scheduler_socket_names': ['/tmp/scheduler94328991'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store49466806', manager_name='/tmp/plasma_manager5285692', manager_port=44591)],
 'redis_address': '127.0.0.1:28963',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui44856.ipynb?token=b6270b2019013e9740412c7ef31ccf7c6ad9ad7b147bc2f5'}

## Immutable remote objects

In Ray, we can create and compute on objects. We refer to these objects as $ \bf \text{remote objects}$, and we use $\bf \text{object IDs}$ to refer to them. Remote objects are stored in $\bf \text{object stores}$, and there is $\bf \text{one}$ object store $\bf \text{per node}$ in the cluster. In the cluster setting, we may $\bf \text{not}$ actually know which machine each object lives on.

An $\bf \text{object ID}$ is essentially a unique ID that can be used to refer to a remote object. If you’re familiar with Futures, our object IDs are conceptually similar.

We assume that remote objects are $\bf \text{immutable}$. That is, their values cannot be changed after creation. $\it\text{This allows remote objects to be replicated in multiple object stores without}$$\it\text{needing to synchronize the copies}$.

### Put and Get

The commands ray.get and ray.put can be used to convert between Python objects and object IDs, as shown in the example below.

In [2]:
x = "example"
ray.put(x)   # Object ID

ObjectID(a5b07911c7d5ec2972539c8bdfa7d33a97fc316f)

The command ray.put(x) would be run by a worker process or by the driver process (the driver process is the one running your script). It takes a Python object and copies it to the local object store (here local means on the same node). Once the object has been stored in the object store, its value cannot be changed.

In addition, ray.put(x) returns an object ID, which is essentially an ID that can be used to refer to the newly created remote object. If we save the object ID in a variable with x_id = ray.put(x), then we can pass x_id into remote functions, and those remote functions will operate on the corresponding remote object.

The command ray.get(x_id) takes an object ID and creates a Python object from the corresponding remote object. For some objects like arrays, we can use shared memory and avoid copying the object. For other objects, this copies the object from the object store to the worker process’s heap. If the remote object corresponding to the object ID x_id does not live on the same node as the worker that calls ray.get(x_id), then the remote object will first be transferred from an object store that has it to the object store that needs it.

In [3]:
x_id = ray.put("example")
ray.get(x_id)   # "example"

'example'

If the remote object corresponding to the object ID x_id has not been created yet, the command ray.get(x_id) will wait until the remote object has been created.

A very common use case of ray.get is to get a list of object IDs. In this case, you can call ray. get(object_ids) where object_ids is a list of object IDs.

In [4]:
result_ids = [ray.put(i) for i in range(10)] 
ray.get(result_ids) # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## Asynchronous Computation in Ray

Ray enables arbitrary Python functions to be executed asynchronously. This is done by designating a Python function as a $\bf \text{remote function}$.

For example, a normal Python function looks like this.

In [5]:
def add1(a, b):
    return a + b

A remote function looks like this.

In [6]:
@ray.remote
def add2(a, b):
    return a + b

### Remote functions

Whereas calling add1(1,2) returns 3 and causes the Python interpreter to block until the computation has finished, calling add2.remote(1, 2) immediately returns an object ID and creates a task. The task will be scheduled by the system and executed asynchronously (potentially on a different machine). When the task finishes executing, its return value will be stored in the object store.

In [7]:
x_id = add2.remote(1, 2)
ray.get(x_id)  # 3

3

The following simple example demonstrates how asynchronous tasks can be used to parallelize computation.

In [8]:
import time

def f1():
    time.sleep(1)
    
@ray.remote
def f2():
    time.sleep(1)
    
# The following takes ten seconds.
[f1() for _ in range(10)]

# The following takes one second (assuming the system has at least ten CPUs).
ray.get([f2.remote() for _ in range(10)])

[None, None, None, None, None, None, None, None, None, None]

There is a sharp distinction between $\it \text{submitting a task}$ and $\it \text{executing the task}$. When a remote function is called, the task of executing that function is $\bf \text{submitted to a local scheduler}$, and object IDs for the outputs of the task are immediately returned. However, the task will not be executed until the system actually schedules the task on a worker. Task execution is not done lazily. The system moves the input data to the task, and the task will execute $\bf \text{as soon as}$ its input dependencies are available and there are enough resources for the computation.

$\bf \text{When a task is submitted, each argument may be passed in by value or by object ID}$. For example, these lines have the same behavior.

In [9]:
add2.remote(1, 2)
add2.remote(1, ray.put(2))
add2.remote(ray.put(1), ray.put(2))

ObjectID(494a4d6b5664c7592e4b8504bb60b73bc48fdbb8)

Remote functions $\bf \text{never}$ return actual values, they always return object IDs.

When the remote function is actually executed, it operates on $\bf \text{Python objects}$. That is, if the remote function was called with any object IDs, the system will retrieve the corresponding objects from the object store.

Note that a remote function can return multiple object IDs.

In [10]:
@ray.remote(num_return_vals=3)
def return_multiple():
    return 1, 2, 3

a_id, b_id, c_id = return_multiple.remote()

### Expressing dependencies between tasks

Programmers can express dependencies between tasks by passing the object ID output of one task as an argument to another task. For example, we can launch three tasks as follows, each of which depends on the previous task.

In [11]:
@ray.remote
def f(x):
    return x + 1

x = f.remote(0)
y = f.remote(x)
z = f.remote(y)
ray.get(z)  # 3

3

The second task above will not execute until the first has finished, and the third will not execute until the second has finished. In this example, there are no opportunities for parallelism.

The ability to compose tasks makes it easy to express interesting dependencies. Consider the following implementation of a tree reduce.

In [13]:
import numpy as np

@ray.remote
def generate_data():
    return np.random.normal(size=1000)

@ray.remote
def aggregate_data(x, y):
    return x + y

# Generate some random data. This launches 100 tasks that will be scheduled
# on various nodes. The resulting data will be distributed around the 
# cluster.
data = [generate_data.remote() for _ in range(100)]

# Perform a tree reduce.
while len(data) > 1:
    data.append(aggregate_data.remote(data.pop(0), data.pop(0)))
                
# Fetch the result.
ray.get(data)

[array([  1.11008555e+01,  -2.13940370e+00,   8.99846779e+00,
          2.45649870e+00,  -1.23372043e+01,  -4.68071812e-01,
         -8.61800500e+00,   4.72110658e+00,  -9.06703671e+00,
          8.73236767e+00,   2.53334744e+00,   7.57010574e+00,
          4.47485786e+00,   9.09002021e-01,   5.81313326e+00,
         -1.81038109e+01,  -3.46480087e+00,  -2.29575880e+00,
         -1.09482880e+01,   7.32369017e+00,   6.32362542e+00,
         -1.19661848e+01,   3.43090236e-01,   6.32923335e+00,
         -1.74437100e+01,  -5.31133352e+00,   1.82246429e+00,
          6.96461496e+00,   1.85737933e+01,  -1.98871228e+00,
          1.26901825e+01,  -1.06945691e-01,  -1.94269066e+00,
         -1.81334111e+01,  -1.41683547e+00,   7.90486496e+00,
          4.07270980e+00,   7.03705479e+00,  -1.31509381e+01,
         -2.28156715e+00,   1.25994179e+01,   2.53749544e+00,
          9.03084900e-01,  -8.44614160e+00,   1.84376784e+01,
          9.36262534e+00,  -1.05133200e+01,  -1.59718501e+00,
        

### Remote Functions Within Remote Functions

So far, we have been calling remote functions only from the driver. But worker processes can also call remote func- tions. To illustrate this, consider the following example.

In [15]:
@ray.remote
def sub_experiment(i, j):
    # Run the jth sub-experiment for the ith experiment.
    return i + j

@ray.remote
def run_experiment(i):
    sub_results = []
    # Launch tasks to perform 10 sub-experiments in parallel.
    for j in range(10):
        sub_results.append(sub_experiment.remote(i, j))
    # Return the sum of the results of the sub-experiments.
    return sum(ray.get(sub_results))

results = [run_experiment.remote(i) for i in range(5)]
ray.get(results)  # [45, 55, 65, 75, 85]

[45, 55, 65, 75, 85]